<a href="https://colab.research.google.com/github/GerardoMunoz/embedded/blob/main/MultipleSockets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple sockets in Micropython

## `select` and `poll`

(Matrial desarrollado con ayuda de un LLM, está pendiente comprobar las diferencias expuestas en los diferentes microcontroladores)


In Python and MicroPython there are several types of objects that allow data exchange through I/O operations. These objects typically implement methods such as:

```python
def fileno(self):
    ...

def recv(self, ...):
    ...

def send(self, ...):
    ...
```

Examples of such objects include:

* Sockets (Python & MicroPython)

* UART devices (MicroPython on some ports, such as ESP32 — not on Raspberry Pi Pico W)

* Files (regular Python and MicroPython POSIX builds, but not on the Raspberry Pi Pico W firmware)

In particular, on the MicroPython implementation for the Raspberry Pi Pico W family, only socket objects provide a `fileno()` method and can be used with the `select` module.

Some operating systems expose a `select` function to monitor these I/O objects for readiness (e.g., when they are ready to read or write). MicroPython wraps this functionality with:

| Name                                     | Description                 |
| ---------------------------------------- | --------------------------- |
| `select(rlist, wlist, xlist[, timeout])` | Wait for readiness on lists |
| `poll()`                                 | Create a Poll object        |


The `Poll` class provides the following methods:

| Method                     | Description                             |
| -------------------------- | --------------------------------------- |
| `register(obj, eventmask)` | Register an I/O object for polling      |
| `unregister(obj)`          | Remove object from polling              |
| `modify(obj, eventmask)`   | Change the event mask                   |
| `poll([timeout])`          | Block and return ready events           |
| `ipoll([timeout])`         | Iterator version (non-blocking polling) |



## `socket`

| **Cliente: Inicio llamada**         | **Cliente: Estado** | **Cliente: Señal** | **Cliente: Resultado/Retorno** | **Servidor: Inicio llamada**       | **Servidor: Estado** | **Servidor: Señal** | **Servidor: Resultado/Retorno**    |
| ----------------------------------- | ------------------- | ------------------ | ------------------------------ | ---------------------------------- | -------------------- | ------------------- | ---------------------------------- |
|                                     |                     |                    |                                | `socket(AF_INET, SOCK_STREAM)`     | CLOSED               |                     |                                    |
|                                     |                     |                    |                                | `bind(("127.0.0.1",4000))`         | LOCAL:127.0.0.1:4000 |                     |                                    |
|                                     |                     |                    |                                | `listen()`                         | LISTEN               |                     |                                    |
|                                     |                     |                    |                                | `accept()`                         | BLOCK                |                     | *(esperando conexión)*             |
| `socket(AF_INET, SOCK_STREAM)`      | CLOSED              |                    |                                |                                    |                      |                     |                                    |
| `connect()`                         | SYN-SENT            | SYN →              |                                |                                    | LISTEN               |                     |                                    |
|                                     |                     | ← SYN+ACK          |                                |                                    | SYN-RECEIVED         | SYN+ACK →           |                                    |
|                                     | ESTABLISHED         | ACK →              | *(conexión establecida)*       | *(UNBLOCK accept())*               | ESTABLISHED          | ACK ←               | `conn, addr = (socket, (ip,port))` |
| `data = bytearray("Hola servidor")` | ESTABLISHED         |                    |                                |                                    | ESTABLISHED          |                     |                                    |
| `s.send(data)`                      | ESTABLISHED         | Data →             |                                |                                    | ESTABLISHED          | ← Data              |                                    |
| `buf = bytearray(4096)`             | ESTABLISHED         |                    |                                |                                    | ESTABLISHED          |                     |                                    |
| `nbytes = s.recv_into(buf)`         | BLOCK               |                    |                                | `recv_into(buf)`                   | BLOCK                | ← Data              |                                    |
| *(UNBLOCK)*                         | ESTABLISHED         |                    | `nbytes, buf`                  | *(UNBLOCK)*                        | ESTABLISHED          |                     | `nbytes, buf`                      |
|                                     |                     |                    |                                | `data = bytearray("Hola cliente")` | ESTABLISHED          |                     |                                    |
|                                     |                     |                    |                                | `s.send(data)`                     | ESTABLISHED          | Data →              |                                    |
| `close()`                           | FIN-WAIT-1          | FIN →              |                                |                                    | CLOSE-WAIT           | ← FIN               |                                    |
|                                     | FIN-WAIT-2          | ← ACK              |                                |                                    | LAST-ACK             | ACK →               | `close()`                          |
|                                     | TIME-WAIT           |                    |                                |                                    | CLOSED               |                     |                                    |
|                                     | CLOSED              |                    |                                |                                    |                      |                     |                                    |


| **Cliente: Inicio llamada**              | **Cliente: Estado** | **Cliente: Señal** | **Cliente: Resultado/Retorno**                      | **Servidor: Inicio llamada**       | **Servidor: Estado** | **Servidor: Señal** | **Servidor: Resultado/Retorno**                        |
| ---------------------------------------- | ------------------- | ------------------ | --------------------------------------------------- | ---------------------------------- | -------------------- | ------------------- | ------------------------------------------------------ |
|                                          |                     |                    |                                                     | `socket(AF_INET, SOCK_STREAM)`     | CLOSED               |                     |                                                        |
|                                          |                     |                    |                                                     | `bind(("127.0.0.1",4000))`         | LOCAL:127.0.0.1:4000 |                     |                                                        |
|                                          |                     |                    |                                                     | `listen()`                         | LISTEN               |                     |                                                        |
|                                          |                     |                    |                                                     | `accept()` (non-blocking) ♻️       | LISTEN               |                     | *(devuelve `BlockingIOError` hasta que haya conexión)* |
| `socket(AF_INET, SOCK_STREAM)`           | CLOSED              |                    |                                                     |                                    |                      |                     |                                                        |
| `connect()` (non-blocking)               | SYN-SENT            | SYN →              | *(puede lanzar `BlockingIOError`, se reintenta ♻️)* |                                    | LISTEN               |                     |                                                        |
|                                          |                     | ← SYN+ACK          |                                                     |                                    | SYN-RECEIVED         | SYN+ACK →           |                                                        |
|                                          | ESTABLISHED         | ACK →              | *(conexión establecida)*                            | `accept()` ♻️                      | ESTABLISHED          | ACK ←               | `conn, addr = (socket, (ip,port))`                     |
| `data = bytearray("Hola servidor")`      | ESTABLISHED         |                    |                                                     |                                    | ESTABLISHED          |                     |                                                        |
| `s.send(data)` (puede no enviar todo) ♻️ | ESTABLISHED         | Data →             | *(usa bucle hasta enviar todo)*                     |                                    | ESTABLISHED          | ← Data              |                                                        |
| `buf = bytearray(4096)`                  | ESTABLISHED         |                    |                                                     |                                    | ESTABLISHED          |                     |                                                        |
| `s.recv_into(buf)` (non-blocking) ♻️     | ESTABLISHED         |                    | *(devuelve 0 o `BlockingIOError` si no hay datos)*  | `recv_into(buf)` (non-blocking) ♻️ | ESTABLISHED          | ← Data              |                                                        |
| *(loop until ready)*                     | ESTABLISHED         |                    | `nbytes, buf`                                       | *(loop until ready)*               | ESTABLISHED          |                     | `nbytes, buf`                                          |
|                                          |                     |                    |                                                     | `data = bytearray("Hola cliente")` | ESTABLISHED          |                     |                                                        |
|                                          |                     |                    |                                                     | `s.send(data)` ♻️                  | ESTABLISHED          | Data →              | *(puede requerir varios envíos)*                       |
| `close()`                                | FIN-WAIT-1          | FIN →              |                                                     |                                    | CLOSE-WAIT           | ← FIN               |                                                        |
|                                          | FIN-WAIT-2          | ← ACK              |                                                     |                                    | LAST-ACK             | ACK →               | `close()`                                              |
|                                          | TIME-WAIT           |                    |                                                     |                                    | CLOSED               |                     |                                                        |
|                                          | CLOSED              |                    |                                                     |                                    |                      |                     |                                                        |


## Módulo uselect.poll() — eventos disponibles

En MicroPython (y CPython también), el método poll() devuelve una lista de tuplas:

events = poller.poll(timeout_ms)
for sock, event in events:
    ...


El valor de event es un bitmask (suma de bits) que indica qué tipo de evento ocurrió en ese socket.

### Principales constantes de eventos (uselect)

| Constante                           | Valor típico | Significado                                               |
| ----------------------------------- | ------------ | --------------------------------------------------------- |
| `uselect.POLLIN`                    | 0x0001       | El socket tiene datos disponibles para leer.              |
| `uselect.POLLOUT`                   | 0x0004       | El socket está listo para enviar datos (no se bloqueará). |
| `uselect.POLLERR`                   | 0x0008       | Error de I/O en el socket.                                |
| `uselect.POLLHUP`                   | 0x0010       | El otro extremo cerró la conexión (Hang Up).              |
| `uselect.POLLNVAL`                  | 0x0020       | Socket no válido (cerrado o no inicializado).             |
| `uselect.POLLEXT` *(algunos ports)* | —            | Extensión específica del puerto (raro).                   |


## Micropython cliente + servidor
```python

import network
import time

SSID = "Ejemplo"
PASSWORD = "12345678"
HOST = "192.168.1.3"   Cambiar el IP y borrar este mensaje  # IP de tu PC
HTTP_PORT = 9001

# -------------------------------------------------------


# Conectar a WiFi
wlan = network.WLAN(network.STA_IF)
wlan.active(True)
if not wlan.isconnected():
    print("Conectando a WiFi...")
    wlan.connect(SSID, PASSWORD)
    timeout = time.time() + 15
    while not wlan.isconnected():
        if time.time() > timeout:
            raise RuntimeError("No se pudo conectar a WiFi")
        time.sleep(0.5)
print("Conectado, IP:", wlan.ifconfig()[0])




import usocket as socket
import uselect
import time

poller = uselect.poll()

# ============================
# Servidor TCP MicroPython
# ============================
srv = socket.socket()
srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
srv.bind(("0.0.0.0", 9000))
srv.listen(1)
srv.setblocking(False)

poller.register(srv, uselect.POLLIN)

server_client = None   # conexión entrante


# ============================
# Cliente TCP MicroPython
# ============================
cli = socket.socket()
cli.setblocking(False)

try:
    cli.connect((HOST, HTTP_PORT))  
except OSError:
    # EINPROGRESS está bien en no-bloqueante
    pass

poller.register(cli, uselect.POLLIN)


# ============================
# Loop principal
# ============================
print("Servidor en :9000, cliente conectando a :9001 ...")

while True:
    events = poller.poll(50)  #50ms

    for sock, evt in events:

        # ---------------------
        # 1. Servidor: aceptar cliente
        # ---------------------
        if sock is srv:
            conn, addr = srv.accept()
            print("Cliente conectado al servidor:", addr)
            conn.setblocking(False)
            server_client = conn
            poller.register(conn, uselect.POLLIN)
            continue

        # ---------------------
        # 2. Servidor: recibir datos
        # ---------------------
        if sock is server_client:
            try:
                data = sock.recv(1024)
            except:
                data = None

            if data:
                print("Servidor recibió:", data)
                sock.send(b"Echo:" + data)
            else:
                print("Cliente servidor desconectado")
                poller.unregister(sock)
                sock.close()
                server_client = None
            continue

        # ---------------------
        # 3. Cliente: recibir datos del servidor Python
        # ---------------------
        if sock is cli:
            try:
                data = cli.recv(1024)
            except:
                data = None

            if data:
                print("Cliente recibió:", data)
            continue

    # ---------------------
    # 4. Cliente: enviar cada segundo
    # ---------------------
    try:
        cli.send(b"Hola desde MicroPython\n")
    except:
        pass

    time.sleep(1)


```

Python Servidor
```python
import socket

HOST = "0.0.0.0"
PORT = 9001

s = socket.socket()
s.bind((HOST, PORT))
s.listen(1)

print("Servidor PC en puerto", PORT)
conn, addr = s.accept()
print("Conectado con:", addr)

while True:
    data = conn.recv(1024)
    if not data:
        break
    print("PC recibió:", data)
    conn.send(b"PC responde: " + data)

```

Python Cliente
```python
import socket

HOST = "192.168.1.57"    Cambiar el IP y borrar este mensaje # IP de tu MicroPython
PORT = 9000

s = socket.socket()
s.connect((HOST, PORT))

print("Cliente PC conectado al MicroPython")

while True:
    msg = input("Mensaje: ").encode()
    s.send(msg)
    resp = s.recv(1024)
    print("MicroPython respondió:", resp)


```